In [1]:
import torch

<div class="alert alert-block alert-info">
<b>Self Attention WIth Trainable Weights</b> 
<p>    

After token embedding , we are able to project the tokens (words) in this sentence into higher dimension vector space.

We can build and understand the semantics and meaning of the individual works .
However the problem arise , we are still not able to build the relationship or context of particular word with the overall sentence /paragraph 

This is why we need attention mechanism , to build the relationship of each token with each other token . meaning how much attention has to paid to other tokens with respect the one in question

Last time we saw the simplified ateention mechanism.
Now in this notebook , we extend the same attention mechanism with trainable Weights. These Weights will be trained and optimized during pre-training the LLM/

In this as compared to previoue , we introduce 3 weight matrices know as
1. Query :- Wq
2. Key :- Wk
3. Value:- Wv

Lets see below what it does
</p>
</div>

In [18]:
#Image this is the Token embedding for the input sentence = "your joruney starts with one step"

inputs = torch.tensor(
 [[0.43, 0.15, 0.89],  # Your
 [0.55, 0.87, 0.66],  # journey
 [0.57, 0.85, 0.64],  # starts
 [0.22, 0.58, 0.33],  # with
 [0.77, 0.25, 0.10],  # one
 [0.05, 0.80, 0.55]] # step
)

In [19]:
# Initials someabs

test_token = inputs[1] # lets pick journey token to see how this works
d_in = inputs.shape[1]
d_out=2

<div class="alert alert-block alert-info">
<b>Initial the Weight Matrices</b> 
<p>    

In self attention mechanism  , we introduce 3 weight matrices know as
1. Query :- Wq
2. Key :- Wk
3. Value:- Wv

Lets initialize them with some random values
</p>
</div>

In [8]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad=False)

-----

<font color='blue'>
    
This is for the Token 2
</font>

In [10]:
# Perform matric multiplication of these initializated Weight Matrix
# We will do it for 2nd Token that is journey
# this is to generated the new matrix query , key and value for 2nd token

query_token_2 = test_token @ W_query
key_token_2 = test_token @ W_key
value_token_2 = test_token @ W_value

In [11]:
print("Query Matrix:",query_token_2)
print("Key Matrix:",key_token_2)
print("Value Matrix:",value_token_2)

Query Matrix: tensor([0.4306, 1.4551])
Key Matrix: tensor([0.4433, 1.1419])
Value Matrix: tensor([0.3951, 1.0037])


---
<font color='blue'>
Lets do it fore all the Tokens

We create the query , key , value matrix for all the input tokens

</font>

In [12]:
query = inputs @ W_query
key   = inputs @ W_key
value = inputs @ W_value

In [13]:
print("Query Matrix:",query.shape)
print("Key Matrix:",key.shape)
print("Value Matrix:",value.shape)

Query Matrix: torch.Size([6, 2])
Key Matrix: torch.Size([6, 2])
Value Matrix: torch.Size([6, 2])


<div class="alert alert-block alert-info">
<b>Calculate Attention Score</b> 
<p>    

Now we calculate the attention score for each token , to see how they relate to the other tokens in the sentence

Basically the attention score matrix looks like below

token1 = [val1, val2, val3, val4, val5, val6]

token2 = [val1, val2, val3, val4, val5, val6]

token3 = [val1, val2, val3, val4, val5, val6]

token3 = [val1, val2, val3, val4, val5, val6]


The columns again map to same number of tokens.
This is like a corelation matrix between each tokens . 
Higher the value more they corelate. 
[0,0] will have high corelation becuase it is between same token1 .

Likewize all the diagonal values will be highly corelated

However in this we calculate the attention score using query matrix and key amtrix for each token
</p>
</div>

In [16]:
# lets calculate the attention score for all tokens
# Dimension for query and ket is 3 * 2 

attention_score = query @ key.T
print(attention_score)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


<div class="alert alert-block alert-info">
<b>Normalize the  Attention Score and divide by sqrt of </b> 
<p>    

The attention scores , needs to be interpreted in probabilities . Meaning the values of attention score for each token should be between 0 and 1 , and should be able to tell the weightage to be give in %

<font color=red> IMP** 
As compared to the  previous simplified self attention method , this method of nomralization has additional step of dividing the attention scores by the square root of embedding dimension of keys
</font>

This is the reason it is called scaled self attentions

</p>
</div>

In [46]:
# Lets perform normalization and also divid the attention score as said earlier

keys_dimension = key.shape[1]
attention_score_norm_step1 = attention_score / keys_dimension**0.5
attention_score_norm = torch.softmax(attention_score_norm_step1 , dim=1)

In [47]:
attention_score_norm

tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])

<div class="alert alert-block alert-info">
<b>Why Divide by sqrt </b> 
<p>    

<font color=red> IMP**

1. softmax kinda gives peak value for higher dot product
2. srqt allow to kinf the variance close to 1 , when the dimensions increase

check :- youtube 
:- https://www.youtube.com/watch?v=UjdRN80c6p8&list=PLPTV0NXA_ZSgsLAr8YCgCwhPIJNNtexWu&index=15 for more details at time 44:45
</font>
</p>
</div>

----

In [48]:
context_vector = attention_score_norm @ value

In [49]:
context_vector

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])

---

<div class="alert alert-block alert-info">
<b>Self Attention Python CLass</b> 
<p>    
Let pack this in a python class.
    It make it simple
</p>
</div>

In [3]:
import torch.nn as nn

In [20]:
class SelfAttention_v1(nn.Module):
    def __init__(self, d_int , d_out):
        super().__init__()

        # Step 1 :- Define the query , key and valye matrices with randowm values
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))

    def forward(self, x):

        # STep2 : create the key , queries and values by multiplying token embedding
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        # Step 3 : Create attention scores 
        attention_scores = queries @ keys.T

        # Step 4 :- Normalize the attention scores by diding then with sqrt od dims and softmax
        attention_scores_norm = torch.softmax(
              attention_scores / keys.shape[1]**0.5 , dim=-1
            )
        
        # Step 5: Create context vector
        context_vector = attention_scores_norm @ values
        return context_vector
    

In [28]:
# Define the shape and size of query , keys and values matrices

torch.manual_seed(123)
d_in = inputs.shape[1]
d_out=2

sa_v1= SelfAttention_v1(d_in , d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<div class="alert alert-block alert-info">
<b>Self Attention Python Class Version 2</b> 
<p>    
We used the <font color=red> nn.parameters </font> to initial the weights for Query, key, and Values matrices.
    However the more preferred way is to use the <font color=red> Linear function</font>. 
    Below are some of the benefits of using Linear over parameters

Using nn.Linear over nn.Parameter provides several benefits:


1. Automatic weight and bias initialization: nn.Linear initializes weights and biases using best practices, reducing manual errors.
2. Built-in forward computation: It handles matrix multiplication and bias addition, simplifying your code.
3. Parameter management: Parameters are automatically registered for optimization and device management.
4. Consistency: Using standard layers makes your code more readable and maintainable.
Integration: Works seamlessly with other PyTorch modules and utilities (e.g., model saving/loading).

Overall, nn.Linear is more convenient and less error-prone for defining learnable linear transformations.
    
</p>
</div>

In [44]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_int , d_out, qkv_bias=False):
        super().__init__()

        # Step 1 :- Define the query , key and valye matrices with randowm values
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)

    def forward(self, x):

        # STep2 : create the key , queries and values by multiplying token embedding
        keys    = self.W_key(x)
        queries = self.W_query(x)
        values  = self.W_value(x)

        # Step 3 : Create attention scores 
        attention_scores = queries @ keys.T

        # Step 4 :- Normalize the attention scores by diding then with sqrt od dims and softmax
        attention_scores_norm = torch.softmax(
              attention_scores / keys.shape[-1]**0.5 , dim=-1
            )
        
        # Step 5: Create context vector
        context_vector = attention_scores_norm @ values
        return context_vector
    

In [45]:
# Define the shape and size of query , keys and values matrices

torch.manual_seed(789)
d_in = inputs.shape[1]
d_out=2

sa_v2= SelfAttention_v2(d_in , d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


<div class="alert alert-block alert-info">
<b>Why QUERY, KEY and VALUE</b> 
<p>    
Consider below analogy

QUERY :- It is pretty much like query in a database. It is the current token the model focus on


KEY :- Each item in input sequence has a KEY . This KEY are ysed to match with the query

VALUE :- Once the KEY is identified , we retrive the value associated with the KEY that is most relevant to the query
    
</p>
</div>